# Converting a Sparse Vector to Columns 

## Often you get files in LIBSVM format but you may need to do EDA on the data, for that feature column, which has sparse vectors, would need to be put into different columns

In [2]:
#Importing Libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import DoubleType, DateType, StringType, BooleanType, IntegerType
import pyspark.sql.functions as F
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors, _convert_to_vector, VectorUDT, DenseVector

In [6]:
spark = SparkSession.builder.appName('fraud').getOrCreate()

In [4]:
#reading data 
path = #put your file path here
df = spark.read.format("libsvm").load(path)

In [6]:
#defining a udf that converts spare vector to dense vector
def sparse_to_dense(vector):
    return DenseVector(vector)

to_dense = udf(sparse_to_dense, VectorUDT())

In [ ]:
#making a df with a dense feature vector
df_dense = df.withColumn('features_dense', to_dense(F.col('features')))

In [8]:
#defining a udf that picks returns the element that is specified
def vec_to_elements(vector, n):
    try:
        return float(vector[n])
    except ValueError:
        return None
to_elements = udf(vec_to_elements, DoubleType())

In [9]:
#checking if the function is working
df_dense.select(to_elements('features_dense',F.lit(0))).show()

+----------------------------+
|vec_to_elements(features, 0)|
+----------------------------+
|                      168.75|
|                        52.0|
|          32.400001525878906|
|           136.9499969482422|
|          157.60000610351562|
|          150.94000244140625|
|           77.37999725341797|
|           58.38999938964844|
|           82.95999908447266|
|          45.869998931884766|
|            38.2400016784668|
|          38.220001220703125|
|           40.97999954223633|
|          120.31999969482422|
|          23.020000457763672|
|            345.489990234375|
|           195.9600067138672|
|          21.190000534057617|
|          253.75999450683594|
|           199.1199951171875|
+----------------------------+
only showing top 20 rows



In [4]:
#put your column names here
col_names = [#colnames]

['total_after_tax',
 'ship_add_lat',
 'ship_add_lon',
 'ship_add_dist2billadd',
 'ship_add_dist2ipadd',
 'ship_add_tzoffset',
 'ship_add_age',
 'ship_add_cust_cnt',
 'ship_add_app_cnt',
 'ship_add_man_app_cnt',
 'ship_add_man_can_cnt',
 'ship_add_man_rej_cnt',
 'ship_add_wpp_is_valid',
 'ship_add_wpp_is_active',
 'ship_add_wpp_is_commercial',
 'ship_add_wpp_is_forwarder',
 'ship_ph_age',
 'ship_ph_cust_cnt',
 'ship_ph_app_cnt',
 'ship_ph_man_app_cnt',
 'ship_ph_man_can_cnt',
 'ship_ph_man_rej_cnt',
 'ship_ph_wpp_is_valid',
 'ship_ph_wpp_is_commercial',
 'proxy_pickup',
 'failed_card_auths',
 'pmt_auth_decline',
 'pmt_cc_age',
 'pmt_cc_cust_cnt',
 'pmt_cc_app_cnt',
 'pmt_cc_man_app_cnt',
 'pmt_cc_man_can_cnt',
 'pmt_cc_man_rej_cnt',
 'bill_add_lat',
 'bill_add_lon',
 'bill_add_dist2ipadd',
 'bill_add_tzoffset',
 'bill_add_age',
 'bill_add_cust_cnt',
 'bill_add_app_cnt',
 'bill_add_man_app_cnt',
 'bill_add_man_can_cnt',
 'bill_add_man_rej_cnt',
 'bill_add_wpp_is_valid',
 'bill_add_wpp_is

In [11]:
#creating a new df with the the feature_dense vector elements put into columns
df_new = df_dense
for i in list(enumerate(col_names)):
    df_new = df_new.withColumn(i[1],to_elements(F.col('features_dense'),F.lit(i[0])))
    

In [12]:
df_new.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- total_after_tax: double (nullable = true)
 |-- ship_add_lat: double (nullable = true)
 |-- ship_add_lon: double (nullable = true)
 |-- ship_add_dist2billadd: double (nullable = true)
 |-- ship_add_dist2ipadd: double (nullable = true)
 |-- ship_add_tzoffset: double (nullable = true)
 |-- ship_add_age: double (nullable = true)
 |-- ship_add_cust_cnt: double (nullable = true)
 |-- ship_add_app_cnt: double (nullable = true)
 |-- ship_add_man_app_cnt: double (nullable = true)
 |-- ship_add_man_can_cnt: double (nullable = true)
 |-- ship_add_man_rej_cnt: double (nullable = true)
 |-- ship_add_wpp_is_valid: double (nullable = true)
 |-- ship_add_wpp_is_active: double (nullable = true)
 |-- ship_add_wpp_is_commercial: double (nullable = true)
 |-- ship_add_wpp_is_forwarder: double (nullable = true)
 |-- ship_ph_age: double (nullable = true)
 |-- ship_ph_cust_cnt: double (nullable = true)
 |-- ship_ph_app_cnt